In [ ]:

# 
import geopandas as gpd


# find geometry intersections within the same dataset using geopandas
def find_intersections(gdf: gpd.GeoDataFrame):
    # Save geometries to another field
    gdf['geom'] = gdf.geometry

    # Self join
    sj = gpd.sjoin(gdf, gdf,
                   how="inner",
                   predicate="intersects",
                   lsuffix="left",
                   rsuffix="right")

    # Remove geometries that intersect themselves
    sj = sj[sj.index != sj.index_right]

    # Extract the intersecting geometry
    sj['intersection_geom'] = sj['geom_left'].intersection(sj['geom_right'])

    # Reset the geometry (remember to set the CRS correctly!)
    sj.set_geometry('intersection_geom', drop=True, inplace=True, crs=gdf.crs)

    # Drop duplicate geometries
    final_gdf = sj.drop_duplicates(subset=['geometry']).reset_index()

    # Drop intermediate fields
    drops = ['geom_left', 'geom_right', 'index_right', 'index']
    final_gdf = final_gdf.drop(drops, axis=1)

    return final_gdf


vector_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Richland_roads.gpkg"
gdf = gpd.read_file(vector_file)
gdf

In [ ]:
inter_gdf = find_intersections(gdf)

In [ ]:
inter_gdf.to_file(r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Richland_roads_intersection.gpkg', layer="intersection", driver="GPKG")

In [ ]:
inter_gdf

In [ ]:
inter_6569_gdf = inter_gdf.to_crs(6569)
inter_6569_gdf

In [ ]:
# inter_6569_gdf.type

In [ ]:
import geopandas as gpd

def buffer_and_dissolve_gdf(gdf, buffer_distance, group_by=None):
    """
    Buffer a point GeoDataFrame and dissolve the resulting buffers.

    Parameters:
        gdf (gpd.GeoDataFrame): Input GeoDataFrame with point geometries.
        buffer_distance (float): The buffer distance in the same units as the GeoDataFrame's CRS.
        group_by (str, optional): Column name to group buffers by for dissolving.
                                  If None, all buffers are dissolved into a single geometry.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame with the dissolved buffers.
    """
    # Ensure the GeoDataFrame contains point geometries
    # if not all(gdf.geometry.type.isin(['Point', 'MultiPoint'])):
        # raise ValueError("Input GeoDataFrame must contain point geometries.")

    # Apply the buffer
    gdf["geometry"] = gdf.geometry.buffer(buffer_distance)

    # Dissolve buffers
    if group_by:
        dissolved_gdf = gdf.dissolve(by=group_by).explode()
    else:
        dissolved_gdf = gdf.dissolve().explode()

    return dissolved_gdf


buffered_gdf = buffer_and_dissolve_gdf(inter_6569_gdf, buffer_distance=30)
buffered_gdf

In [ ]:
# buffered_gdf.explode()
buffered_gdf.to_file(r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Richland_roads_intersection_buffer.gpkg', layer="intersection", driver="GPKG")

In [ ]:
buffered_gdf.explore() 

In [ ]:
# Downlad Epicollection photos

In [1]:
! pip install pyepicollect

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyepicollect: filename=pyepicollect-5.1.1-py3-none-any.whl size=6575 sha256=ce4cb9172ccc05ea492ac9a8a3592a7fe2f3961d04281794118b4dd121bf907a
  Stored in directory: c:\users\n\appdata\local\pip\cache\wheels\1b\2b\84\61623e3c09ca89c2b0aa160426feb16e59ded8bf3c5d372b62
Successfully built pyepicollect


    keyring (<=21.8.*,>=19)
             ~~~~~~~^

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import requests
import os

# Define the path to your CSV file
csv_file_path = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_St_ground_truth\form-1__tree.csv"

# Define the directory where images will be saved
output_directory = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_St_ground_truth\photos"
os.makedirs(output_directory, exist_ok=True)

# Define the base URL of the endpoint
base_url = "https://five.epicollect.net/api/internal/media/tree-diameter-measurement?type=photo&format=entry_original&name="

# Load the CSV file
df = pd.read_csv(csv_file_path)

# Extract the image basenames from the column "5_Photo"
image_basenames = df["5_Photo"]

# Function to download an image from the constructed URL
def download_image(image_basename):
    # Construct the full image URL
    image_url = base_url + image_basename
    try:
        response = requests.get(image_url)
        print(image_url)
        print(response)
        response.raise_for_status()  # Check if the request was successful
        print(response)
        # Save the image
        image_path = os.path.join(output_directory, image_basename)
        with open(image_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {image_basename}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {image_basename}: {e}")

# Loop through the image basenames and download each image
for basename in image_basenames:
    download_image(basename)


In [ ]:
import requests
import json

# Define the endpoint and credentials
token_url = 'https://five.epicollect.net/api/oauth/token'
client_id = '5672'  # Replace with your actual Client ID
client_secret = '4cnBOLLLojRALnOCUB0DTVEk076mS7dvtoPY5TRI'  # Replace with your actual Client Secret

# Define the parameters
params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

# Send the POST request
response = requests.post(token_url, json=params, headers={'Content-Type': 'application/json'})

# Check if the request was successful
if response.status_code == 200:
    # Parse the response
    token_info = response.json()
    access_token = token_info['access_token']
    print(f"Access Token: {access_token}")
else:
    print(f"Failed to retrieve token: {response.status_code}")
    print(response.text)


In [ ]:
import requests
import os

# Define the project slug (replace 'ec5-api-test' with your project slug)
project_slug = 'tree-diameter-measurement'
base_api_url = f'https://five.epicollect.net/api/export/entries/{project_slug}'

# Define the media endpoint base
media_base_url = f'https://five.epicollect.net/api/export/media/{project_slug}?type=photo&format=entry_original&name='

# Define the directory where images will be saved
output_directory = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_St_ground_truth\photos"
os.makedirs(output_directory, exist_ok=True)

# Function to download an image given the filename
def download_image(image_name):
    image_url = media_base_url + image_name
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image_path = os.path.join(output_directory, image_name)
        with open(image_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {image_name}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {image_name}: {e}")

# Fetch entries from the project
def fetch_entries():
    try:
        response = requests.get(base_api_url, headers={'Content-Type': 'application/vnd.api+json'})
        response.raise_for_status()
        entries = response.json()['data']['entries']
        return entries
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch entries: {e}")
        return []

# Main function to get entries and download corresponding images
def download_project_images():
    entries = fetch_entries()
    for entry in entries:
        # Assuming the image filename is stored under 'photo', adapt based on actual response
        image_name = entry.get('photo')
        image_name = "test.jpg"
        if image_name:
            download_image(image_name)
        else:
            print(f"No image found for entry {entry['id']}")

# Run the script to download images
if __name__ == "__main__":
    download_project_images()
